### Evaluación peresoza.

La computación paralela utiliza lo que se denomina evaluación "perezosa".

Esto significa que su marco pondrá en cola conjuntos de transformaciones o cálculos para que estén listos para ejecutarse más tarde, en paralelo.

Este es un concepto que encontrará en muchos marcos para computación paralela, incluido Dask.

Su marco no evaluará los cálculos solicitados hasta que se le indique explícitamente.

Esto difiere de las funciones de evaluación "ansiosas", que calculan instantáneamente al ser llamadas.

Muchas funciones muy comunes y útiles están portadas para ser nativas en Dask, lo que significa que serán perezosas (cálculo retrasado) sin que tengas que preguntar.

Sin embargo, a veces tendrá un código personalizado complicado que está escrito en pandas, scikit-learn o incluso base python, que no está disponible de forma nativa en Dask.

Otras veces, es posible que simplemente no tenga el tiempo o la energía para refactorizar su código en Dask, si se necesitan ediciones para aprovechar los elementos nativos de Dask.

Si este es el caso, puede decorar sus funciones con **@dask.delayed**, que establecerá manualmente que la función debe ser perezosa y no evaluará hasta que se lo indique.

Lo diría con los procesos **.compute()** o **.persist()**, descritos en la siguiente sección.

In [12]:
def exponente(x, y): 
    '''Define una función básica.''' 
    return x ** y

# La función devuelve el resultado inmediatamente cuando se llama 
exponente (4, 5)

1024

In [13]:
import dask

@dask.delayed
def lazy_exponent(x, y):
    '''Definir una función de evaluación perezosa''' 
    return x ** y
# La función devuelve un objeto retrasado, no un cálculo 
lazy_exponent(4, 5)

Delayed('lazy_exponent-d2324939-8cd3-490a-beeb-d7dc6336f32d')

In [14]:
# Esto ahora devolverá el cálculo 

lazy_exponent(4,5).compute()

1024

Podemos tomar este conocimiento y expandirlo, debido a que nuestra función perezosa devuelve un objeto, podemos asignarlo y luego encadenarlo de diferentes maneras más adelante.

Aquí devolvemos un valor retrasado de la primera función y lo llamamos x.

Luego pasamos x a la función por segunda vez y la llamamos y.

Finalmente, multiplicamos x e y para producir z.

In [15]:
x = lazy_exponent(4, 5)
y = lazy_exponent(x, 2)
z = x * y
z

Delayed('mul-a92ba319b79bf062c5d7ddb5278f7274')

In [16]:
z.compute()

1073741824

**Persistir vs Calcular**

¿Cómo debemos indicarle a Dask que ejecute los cálculos que hemos puesto en cola con pereza? 

Tenemos dos opciones: *.persist()* y *.compute()*

**Calcular**

Si usamos **.compute()**, le estamos pidiendo a Dask que tome todos los cálculos y ajustes a los datos que hemos puesto en cola, los ejecute y los traiga a la superficie aquí.

Eso significa que si se distribuyó, queremos convertirlo en un objeto local aquí y ahora.

Si es un marco de datos de Dask, cuando llamamos .compute(), decimos "Ejecute las transformaciones que hemos puesto en cola y conviértalo en un marco de datos de pandas de inmediato".

*Sin embargo, tenga cuidado: si su conjunto de datos es extremadamente grande, esto podría significar que no tendrá suficiente memoria para completar esta tarea, ¡y su kernel podría bloquearse!*

**Persistir**

Si usamos **.persist()**, le estamos pidiendo a Dask que tome todos los cálculos y ajustes a los datos que hemos puesto en cola y los ejecute, pero luego el objeto permanecerá distribuido y vivirá en el clúster (un LocalCluster si está en una máquina), no en la instancia de Jupyter u otro entorno local.

Entonces, cuando hacemos esto con un Dask Dataframe, le estamos diciendo a nuestro clúster: "Ejecute las transformaciones que hemos puesto en cola y deje esto como un Dask Dataframe distribuido".

Entonces, si desea procesar todas las tareas retrasadas que ha aplicado a un objeto Dask, cualquiera de estos métodos lo hará. La diferencia es dónde vivirá su objeto al final.

Comprender esto es realmente útil para saber cuándo **.persist()** podría tener sentido.

Considere una situación en la que carga datos y luego los usa para muchas tareas complejas.

Si usa un Dask Dataframe cargado desde CSV en el disco, es posible que desee llamar **.persist()** antes de pasar estos datos a otras tareas, ya que las otras tareas ejecutarán la carga de datos una y otra vez cada vez que se refieran a ellos.

Si usa **.persist()** primero, entonces el paso de carga debe ejecutarse solo una vez.

In [10]:
import dask
import numpy as np
from dask import delayed

@dask.delayed
def inc(x):
    return x + 1

@dask.delayed
def double(x):
    return x * 2

@dask.delayed
def add(x, y):
    return x + y

data = [1, 2, 3, 4, 5]

output = []
for x in data:
    a = inc(x)
    b = double(x)
    c = add(a, b)
    output.append(c)

print(output)
print(output[0].compute())
print(output[1].compute())
print(output[2].compute())
print(output[3].compute())
print(output[4].compute())

total = dask.delayed(sum)(output)
total.compute()

[Delayed('add-598fdc90-48f1-47c0-b9d0-30f71ebcaf71'), Delayed('add-c6d45f8e-ec7f-4fd0-afa6-1def45db7b26'), Delayed('add-7a128c3b-b535-480e-ba48-f4032390f97c'), Delayed('add-060553c4-a909-454d-9029-d496003fb786'), Delayed('add-477cf7ed-9353-4f4a-8e86-3f8e8b92a20a')]
4
7
10
13
16


50

In [18]:
def my_square_function(x):
    return x**2

delayed_square_function = delayed(my_square_function)

delayed_square_function(4).compute()

16

In [23]:
#Uso de los resultados de un decorator delayed()
result_delayed = delayed(my_square_function)(4)

((4 + result_delayed) * 5).compute()

100

In [25]:
x_list = [30, 85, 14, 12, 27, 62, 89, 15, 78,  0]

sum_of_squares = 0

for x in x_list:
    
    sum_of_squares += delayed(my_square_function)(x)

sum_of_squares.compute()

27268

In [4]:
def fraction_to_percent(x):
     percentage = x * 100
     print('Converting to percentage')
     return x

frac = 0.3
percentage = delayed(fraction_to_percent)(frac)
computed_percentage = percentage.compute()

print(percentage)
print(computed_percentage)

Converting to percentage
Delayed('fraction_to_percent-8075248f-fcfa-4d25-8b56-3e5fbfc37742')
0.3


In [15]:
costs_week_1 = [121, 729, 441, 961, 841, 729, 25, 225, 256, 441, 400, 484, 900]
costs_week_2 = [196,361,81,441,49,100,729,841,676,256,121,576,49,100,49,16,961,36,841]

sum1 = delayed(np.sum)(costs_week_1)
sum2 = delayed(np.sum)(costs_week_2)

total = sum1 + sum2

print(total.compute())

13032


**¿Cuáles son los diferentes planificadores en Python?**

Dask le permite usar procesamiento paralelo o subprocesos múltiples.

Cada uno de estos tiene sus ventajas y desventajas, y cuál funciona mejor dependerá de su tarea.

Afortunadamente, Dask facilita el cambio entre ellos, por lo que puede probar ambos, pero para ganar rendimiento, es importante conocer las características de ambos.

Importante el concepto de GIL (bloqueo de intérprete global de python), en palabras simples, permite que solo un subproceso tenga el control del intérprete de python.

Esto significa que solo un subproceso puede estar en estado de ejecución en cualquier momento. El impacto de la GIL no es visible para los desarrolladores que ejecutan programas de subproceso único, pero puede ser un cuello de botella en el rendimiento en el código de subprocesos múltiples y vinculado a la CPU.

Dado que GIL permite que solo se ejecute un subproceso a la vez, incluso en una arquitectura de subprocesos múltiples con más de un núcleo de CPU, GIL se ha ganado la reputación de ser una característica "infame" de Python.

https://realpython.com/python-gil/

**Subprocesos múltiples** : Un subproceso es una entidad dentro de un proceso que se puede programar para su ejecución. Además, es la unidad de procesamiento más pequeña que se puede realizar en un SO (Sistema Operativo)

https://www.geeksforgeeks.org/multithreading-python-set-1/?ref=lbp

In [11]:
import threading

def print_cube(num):
    print("Cube: {}" .format(num * num * num))
  
def print_square(num):
    print("Square: {}" .format(num * num))

if __name__ =="__main__":
    
    t1 = threading.Thread(target=print_square, args=(10,))
    t2 = threading.Thread(target=print_cube, args=(10,))
 
    t1.start()
    t2.start()
 
    t1.join()
    t2.join()
 
    print("Done!")

Square: 100
Cube: 1000
Done!


**Procesamiento en paralelo** : El multiprocesamiento se refiere a la capacidad de un sistema para admitir más de un procesador al mismo tiempo. Las aplicaciones en un sistema de multiprocesamiento se dividen en rutinas más pequeñas que se ejecutan de forma independiente. El sistema operativo asigna estos hilos a los procesadores mejorando el rendimiento del sistema.

https://www.geeksforgeeks.org/multiprocessing-python-set-2/?ref=lbp

In [ ]:
import multiprocessing
  
def print_cube(num):
    """
    function to print cube of given num
    """
    print("Cube: {}".format(num * num * num))
  
def print_square(num):
    """
    function to print square of given num
    """
    print("Square: {}".format(num * num))
  
if __name__ == "__main__":
    #Para crear un proceso, creamos un objeto de la clase Proceso.
    #Toma los siguientes argumentos:
    #target : la función a ser ejecutada por el proceso
    #args : los argumentos que se pasarán a la función de destino
    
    #En el ejemplo anterior, creamos 2 procesos con diferentes funciones objetivo:
    p1 = multiprocessing.Process(target=print_square, args=(10, ))
    p2 = multiprocessing.Process(target=print_cube, args=(10, ))
  
    #Para iniciar un proceso, usamos el método de start()
    p1.start()
    p2.start()
  
    #Una vez que se inician los procesos, el programa actual también sigue ejecutándose.
    #Para detener la ejecución del programa actual hasta que se complete un proceso,
    #usamos el método de unión.
    p1.join()
    p2.join()
  
    print("Done!")

**Objetos de datos distribuidos**

Hay otra área de trabajo de la que debemos hablar, además de retrasar las funciones individuales: estos son los objetos de datos de Dask.

Estos incluyen la bolsa Dask (un objeto paralelo basado en listas), la matriz Dask (un objeto paralelo basado en matrices NumPy) y el Dask Dataframe (un objeto paralelo basado en pandas Dataframes).

Para mostrar lo que estos objetos pueden hacer, discutiremos el Dask Dataframe.

Imagine que tenemos los mismos datos durante varios años y nos gustaría cargarlos todos a la vez.

Podemos hacerlo fácilmente con Dask, y la API es muy parecida a la API de pandas.

Un marco de datos de Dask contiene varios marcos de datos de pandas, que se distribuyen en su clúster.

In [17]:
import dask
import dask.dataframe as dd

df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int32,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [18]:
df2 = df[df.y > 0]
df3 = df2.groupby('name').x.std()
df3

Dask Series Structure:
npartitions=1
    float64
        ...
Name: x, dtype: float64
Dask Name: sqrt, 157 tasks

Esto nos devuelve no una serie de pandas, sino una serie de Dask.

Tiene una partición aquí, por lo que no se divide entre diferentes trabajadores. 

Es bastante pequeño, así que esto está bien para nosotros.

¿Qué pasa si corremos *.compute()* con esto?

In [19]:
computed_df = df3.compute()
type(computed_df)

pandas.core.series.Series

In [20]:
computed_df.head()

name
Alice      0.577623
Bob        0.576359
Charlie    0.575616
Dan        0.578138
Edith      0.576330
Name: x, dtype: float64

Volvamos al objeto df que usamos en el ejemplo anterior.

Aquí vamos a usar la *npartitions* propiedad para verificar en cuántas partes se divide nuestro marco de datos.

In [21]:
df.npartitions

30

In [22]:
df2 = df[df.y > 0]
df3 = df2.groupby('name').x.std()
print(type(df3))
df3.npartitions

<class 'dask.dataframe.core.Series'>


1

In [23]:
df4 = df3.repartition(npartitions=3)
df4.npartitions

3

In [24]:
df4

Dask Series Structure:
npartitions=3
    float64
        ...
        ...
        ...
Name: x, dtype: float64
Dask Name: repartition, 161 tasks

In [25]:
%%time

df4.persist()

Wall time: 830 ms


Dask Series Structure:
npartitions=3
    float64
        ...
        ...
        ...
Name: x, dtype: float64
Dask Name: repartition, 3 tasks

In [26]:
%%time

df4.compute().head()

Wall time: 798 ms


name
Alice      0.577623
Bob        0.576359
Charlie    0.575616
Dan        0.578138
Edith      0.576330
Name: x, dtype: float64

**Conclusión**

¡Con eso, tienes los conceptos básicos que necesitas para usar Dask!

   - Su código personalizado se puede hacer paralelizable con **dask.delayed**

   - El ecosistema de Dask tiene un sólido soporte nativo para las funcionalidades de pandas, NumPy y scikit-learn, lo que les brinda capacidad de paralelización.

   - Los objetos de datos de Dask pueden hacer que sus datos se distribuyan, evitando problemas con demasiados datos o muy poca memoria.
    
Al combinar estas excelentes funciones, puede producir canalizaciones de datos potentes y de calidad de producción con las mismas API que usamos para pandas, NumPy y scikit-learn. 
